In [1]:
import re
import time
import random
from random import randint
from time import sleep
from urllib.request import Request, urlopen
import requests
import pandas as pd
import logging
import sys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import (ElementClickInterceptedException,
                                        ElementNotInteractableException,
                                        NoSuchElementException,
                                        NoSuchWindowException,
                                        WebDriverException)
from selenium.webdriver.common.by import By

In [2]:
logging.basicConfig(format='%(asctimes)|%(levelname)s : %(message)s', 
                   level = logging.INFO, stream=sys.stdout)

In [3]:
pd.set_option('display.max_column',None)
driver = webdriver.Chrome()

In [4]:
# Get one page by URL ( Textbook on Computer Science)
url = 'https://www.amazon.com/s?bbn=465600&rh=n%3A283155%2Cn%3A465600%2Cn%3A468204&dc&qid=1676892646&rnid=465600&ref=lp_465600_nr_n_2'
driver.get(url) 
driver.refresh()
time.sleep(5)
driver.title

'Amazon.com: Computer Science: Books: Programming Languages, Artificial Intelligence & More'

In [146]:
# scroll page 
actions = ActionChains(driver)
for _ in range(2):
    actions.send_keys(Keys.SPACE).perform()
    time.sleep(4)
# click See all results 
see_button = driver.find_element(By.LINK_TEXT, 'See all results')
see_button.click()
driver.refresh()
sleep(random.randint(4,6))

### Get title and detail urls 

In [147]:
def get_data():
    res_title = []
    res_url = []
    while True: 
        try:
            # get titles 
            for i in range(2,19):
                title = driver.find_elements(By.XPATH,'//*[@id="search"]/div[1]/div[1]/div/span[1]/div[1]/div[{}]/div/div/div/div/div/div[2]/div/div/div[1]/h2/a/span'.format(i))
                title = [x.text for x in title]
                res_title += title
            # get all links 
            for i in range(2,19):
                links = driver.find_elements(By.XPATH,'//*[@id="search"]/div[1]/div[1]/div/span[1]/div[1]/div[{}]/div/div/div/div/div/div[2]/div/div/div[1]/h2/a'.format(i))
                links = [x.get_attribute('href') for x in links]
                res_url += links
            
            """Loop to click next page"""
            next_button = driver.find_element(By.CLASS_NAME, 's-pagination-next')
            if(not next_button):
                # Close browser
                driver.close()  
                break
            next_button.click()
            sleep(random.randint(5,8))                        
        except (NoSuchElementException, WebDriverException) as e:
            break
            
    df = pd.DataFrame(list(zip(res_title, res_url)), columns = ['titles', 'urls'])
    return df

In [7]:
cs = get_data()
cs

,titles,urls
0,Cracking the Coding Interview: 189 Programming...,https://www.amazon.com/Cracking-Coding-Intervi...
1,Designing Data-Intensive Applications: The Big...,https://www.amazon.com/Designing-Data-Intensiv...
2,Machine Learning System Design Interview,https://www.amazon.com/Machine-Learning-System...
3,CompTIA A+ Certification All-in-One Exam Guide...,https://www.amazon.com/CompTIA-Certification-E...
4,(ISC)2 CISSP Certified Information Systems Sec...,https://www.amazon.com/Certified-Information-S...
...,...,...
1595,Artificial Intelligence Programming with Pytho...,https://www.amazon.com/Artificial-Intelligence...
1596,Fundamentals of Biostatistics,https://www.amazon.com/Fundamentals-Biostatist...
1597,Applied Predictive Modeling,https://www.amazon.com/Applied-Predictive-Mode...
1598,An Introduction to Mathematical Cryptography (...,https://www.amazon.com/Introduction-Mathematic...


In [10]:
cs.to_csv('cs_textbooks.csv')

In [ ]:
df1 = cs.drop_duplicates(keep='first').reset_index()
print(len(df1))
df1

In [ ]:
df1.to_csv('df1.csv')

### Functions for 1 details url 

In [22]:
#  functions for 1 details url 
# get book_name  
def get_name():
    try: 
        name = driver.find_element(By.XPATH,'//*[@id="productTitle"]').text
    except NoSuchElementException:
        name = 'Na'
    return name
# get author ############
def get_author(): 
    try:
        author = driver.find_element(By.XPATH,'//*[@id="bylineInfo"]/span/a')
    except NoSuchElementException:
        try:
            author = driver.find_element(By.XPATH,'//*[@id="bylineInfo"]/span/span[1]/a[1]')
        except NoSuchElementExeption:
            author = 'Na'
        else:
            author = author.text
    else:
        author = author.text
            
    return author 
# get rating_value 
def get_rating():
    try: 
        rating = driver.find_element(By.XPATH,'//*[@id="reviewsMedley"]/div/div[1]/span[1]/span/div[2]/div/div[2]/div/span/span').text
    except NoSuchElementException:
        rating = 'Na'
    return rating 
# get number of ratings 
def get_rating_num():
    try: 
        rat_num = driver.find_element(By.XPATH, '//*[@id="acrCustomerReviewText"]').text
    except NoSuchElementException:
        rat_num ='Na'
    return rat_num

# get price  ######################
def get_price(): 
    try:
        price = driver.find_element(By.XPATH,'//*[@id="price"]')
    except NoSuchElementException:
        try: 
            price = driver.find_element(By.XPATH,'//*[@id="corePrice_feature_div"]/div/span')
        except NoSuchElementException:
            try: 
                price = driver.find_element(By.XPATH,'//*[@id="price_inside_buybox"]')
            except NoSuchElementException:
                price = 'Na'
            else:
                price = price.text
        else:
            price = price.text
    else:
        price = price.text
    return price
# get price for rent 
def get_price_rent(): 
    try: 
        price_rent = driver.find_element(By.XPATH,'//*[@id="rentPrice"]').text
    except NoSuchElementException:
        price_rent ='Na'
    return price_rent
# get category 
def get_category():
    try:
        category = driver.find_element(By.XPATH, '//*[@id="wayfinding-breadcrumbs_feature_div"]/ul/li[5]/span/a').text
    except NoSuchElementException:
        category = 'Na'
    return category
# get product details 
def product_details(): # //*[@id="detailBullets_feature_div"]/ul
    try:
        details = driver.find_element(By.XPATH, '//*[@id="detailBullets_feature_div"]/ul').text
    except NoSuchElementException:
        details ='Na'
    return details
# get detail of rank 
def best_seller_rank():
    try:
        rank = driver.find_element(By.XPATH, '//*[@id="corePrice_feature_div"]/div/span/span[1]').text
    except NoSuchElementException:
        rank ='Na'
    return rank
# get tag best_seller
def tag_best_seller():
    try: 
        tag = driver.find_element(By.XPATH,'//*[@id="zeitgeistBadge_feature_div"]/div/a/i').text
    except NoSuchElementException:
        tag = 'Na'
    return tag
# get best_seller in categories ?
def categorize_best_seller():
    try: 
        tag_category = driver.find_element(By.XPATH,'//*[@id="zeitgeistBadge_feature_div"]/div/a/span/span').text
    except NoSuchElementException:
        tag_category = 'Na'
    return tag_category
# get subject 
def get_subject():
    try: 
        subject = driver.find_element(By.XPATH,'//*[@id="seriesBulletWidget_feature_div"]/a').text
    except NoSuchElementException:
        subject = 'Na'
    return subject
# # get publisher 
# def get_publisher():
#     try: 
#         publisher = driver.find_element(By.XPATH,'//*[@id="rpi-attribute-book_details-publisher"]/div[3]/span').text
#     except NoSuchElementException:
#         publisher = 'Na'
#     return publisher
# # get publication_date
# def get_publication_date():
#     try: 
#         publication_date = driver.find_element(By.XPATH,'//*[@id="rpi-attribute-book_details-publication_date"]/div[3]/span').text
#     except NoSuchElementException:
#         publication_date = 'Na'
#     return publication_date
    


In [18]:
def get_details(urls):
    res_name = []
    res_author = []
    res_rating = []
    res_rating_num = []
    res_price = []
    res_price_rent = []
    res_category = []
    res_detail = []
    res_rank = []
    res_tag = [] # tag best seller 
    res_tag_in = []
    res_part = []
    
    for url in urls:
        req = Request(url, headers = {'User-Agent': 'Mozilla/5.0'})
        #driver.implicitly_wait(10)
        driver.get(url) 
        driver.refresh()
        sleep(random.randint(4,8))
        # funtions 
        # get book name 
        name = get_name()
        res_name.append(name)
        # get author
        author = get_author()
        res_author.append(author)
        # get rating value 
        rating = get_rating()
        res_rating.append(rating)
        # get rating_num 
        rating_num = get_rating_num()
        res_rating_num.append(rating_num)
        # get price buy new 
        price = get_price()
        res_price.append(price)
        # get price for rent 
        rent = get_price_rent()
        res_price_rent.append(rent)
        # get category 
        cate = get_category()
        res_category.append(cate)
        # get product detail  
        detail = product_details()
        res_detail.append(detail)
        # best seller rank 
        rank = best_seller_rank()
        res_rank.append(rank)
        # get tag 
        tag = tag_best_seller()
        res_tag.append(tag)
        # tag in ?
        tag_in = categorize_best_seller()
        res_tag_in.append(tag_in)
        # part of ?
        part = get_subject()
        res_part.append(part)
    # save to dataframe 
    df = pd.DataFrame(list(zip(res_name, res_author, res_rating, res_rating_num, res_price, res_price_rent, res_category, res_detail,
                               res_rank, res_tag, res_tag_in, res_part)), 
                     columns = ['book_name', 'author', 'rating_value', 'rating_num', 'price_buy_new', 'price_rent', 'category',
                               'details', 'rank','tag', 'tag_in', 'part_in'])
    return df  
        

In [31]:
df1 = pd.read_csv('df1.csv')
df1

,Unnamed: 0,index,titles,urls
0,0,0,Cracking the Coding Interview: 189 Programming...,https://www.amazon.com/Cracking-Coding-Intervi...
1,1,1,Designing Data-Intensive Applications: The Big...,https://www.amazon.com/Designing-Data-Intensiv...
2,2,2,Machine Learning System Design Interview,https://www.amazon.com/Machine-Learning-System...
3,3,3,CompTIA A+ Certification All-in-One Exam Guide...,https://www.amazon.com/CompTIA-Certification-E...
4,4,4,(ISC)2 CISSP Certified Information Systems Sec...,https://www.amazon.com/Certified-Information-S...
...,...,...,...,...
1211,1211,1595,Artificial Intelligence Programming with Pytho...,https://www.amazon.com/Artificial-Intelligence...
1212,1212,1596,Fundamentals of Biostatistics,https://www.amazon.com/Fundamentals-Biostatist...
1213,1213,1597,Applied Predictive Modeling,https://www.amazon.com/Applied-Predictive-Mode...
1214,1214,1598,An Introduction to Mathematical Cryptography (...,https://www.amazon.com/Introduction-Mathematic...


In [8]:
# Function to extract ASIN from URL
def extract_asin(url):
    asin = re.findall(r'/dp/([A-Z0-9]{10})', url)
    return asin[0] if asin else None
# Function to get asin url 
def get_asin_url(asin):
    res = "https://www.amazon.com/dp/" + asin + "/"
    return res 
# Add new column with ASIN
df1['asin'] = df1['urls'].apply(extract_asin)
df1['asin_url'] = df1['asin'].apply(get_asin_url)
df1 = df1.drop_duplicates(subset='asin').reset_index().iloc[:, 2:]

In [9]:
df1

,index,titles,urls,asin,asin_url
0,0,Cracking the Coding Interview: 189 Programming...,https://www.amazon.com/Cracking-Coding-Intervi...,0984782850,https://www.amazon.com/dp/0984782850/
1,1,Designing Data-Intensive Applications: The Big...,https://www.amazon.com/Designing-Data-Intensiv...,1449373321,https://www.amazon.com/dp/1449373321/
2,2,Machine Learning System Design Interview,https://www.amazon.com/Machine-Learning-System...,1736049127,https://www.amazon.com/dp/1736049127/
3,3,CompTIA A+ Certification All-in-One Exam Guide...,https://www.amazon.com/CompTIA-Certification-E...,1264609906,https://www.amazon.com/dp/1264609906/
4,4,(ISC)2 CISSP Certified Information Systems Sec...,https://www.amazon.com/Certified-Information-S...,1119790026,https://www.amazon.com/dp/1119790026/
...,...,...,...,...,...
1191,1195,Artificial Intelligence Programming with Pytho...,https://www.amazon.com/Artificial-Intelligence...,1119820863,https://www.amazon.com/dp/1119820863/
1192,1196,Fundamentals of Biostatistics,https://www.amazon.com/Fundamentals-Biostatist...,130526892X,https://www.amazon.com/dp/130526892X/
1193,1197,Applied Predictive Modeling,https://www.amazon.com/Applied-Predictive-Mode...,1461468485,https://www.amazon.com/dp/1461468485/
1194,1198,An Introduction to Mathematical Cryptography (...,https://www.amazon.com/Introduction-Mathematic...,1493917102,https://www.amazon.com/dp/1493917102/


In [10]:
df1.to_csv('df1_asin.csv')


In [4]:
###########
df1_asin= pd.read_csv('df1_asin.csv')
df1_asin

,Unnamed: 0,index,titles,urls,asin,asin_url
0,0,0,Cracking the Coding Interview: 189 Programming...,https://www.amazon.com/Cracking-Coding-Intervi...,0984782850,https://www.amazon.com/dp/0984782850/
1,1,1,Designing Data-Intensive Applications: The Big...,https://www.amazon.com/Designing-Data-Intensiv...,1449373321,https://www.amazon.com/dp/1449373321/
2,2,2,Machine Learning System Design Interview,https://www.amazon.com/Machine-Learning-System...,1736049127,https://www.amazon.com/dp/1736049127/
3,3,3,CompTIA A+ Certification All-in-One Exam Guide...,https://www.amazon.com/CompTIA-Certification-E...,1264609906,https://www.amazon.com/dp/1264609906/
4,4,4,(ISC)2 CISSP Certified Information Systems Sec...,https://www.amazon.com/Certified-Information-S...,1119790026,https://www.amazon.com/dp/1119790026/
...,...,...,...,...,...,...
1191,1191,1195,Artificial Intelligence Programming with Pytho...,https://www.amazon.com/Artificial-Intelligence...,1119820863,https://www.amazon.com/dp/1119820863/
1192,1192,1196,Fundamentals of Biostatistics,https://www.amazon.com/Fundamentals-Biostatist...,130526892X,https://www.amazon.com/dp/130526892X/
1193,1193,1197,Applied Predictive Modeling,https://www.amazon.com/Applied-Predictive-Mode...,1461468485,https://www.amazon.com/dp/1461468485/
1194,1194,1198,An Introduction to Mathematical Cryptography (...,https://www.amazon.com/Introduction-Mathematic...,1493917102,https://www.amazon.com/dp/1493917102/


In [8]:
d1 = df1_asin['asin_url'][:400]
d2 = df1_asin['asin_url'][400:800]
d3 = df1_asin['asin_url'][800:]

In [23]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

In [24]:
result_1 = get_details(d1)
result_1.to_csv('result_1.csv')
driver.quit()
result_1

,book_name,author,rating_value,rating_num,price_buy_new,price_rent,category,details,rank,tag,tag_in,part_in
0,Cracking the Coding Interview: 189 Programming...,Gayle Laakmann McDowell,4.7 out of 5,"8,162 ratings",$35.95,Na,Programming,ASIN : 0984782850\nPublisher : CareerCup; 6th ...,,#1 Best Seller,in Job Interviewing,Part of: Cracking the Interview & Career (3 bo...
1,Designing Data-Intensive Applications: The Big...,Martin Kleppmann,4.8 out of 5,"3,663 ratings",$47.94,Na,Databases & Big Data,Publisher : O'Reilly Media; 1st edition (May 2...,Na,#1 Best Seller,in MySQL Guides,Na
2,Machine Learning System Design Interview,Ali Aminian,4.5 out of 5,10 ratings,$38.00,Na,Computer Science,"Publisher : ByeByteGo (January 28, 2023)\nLang...",Na,#1 Best Seller,in Computer Science,Na
3,CompTIA A+ Certification All-in-One Exam Guide...,Travis Everett,4.3 out of 5,23 ratings,$54.00,Na,Hardware & DIY,Publisher : McGraw Hill; 11th edition (January...,Na,#1 Best Seller,in CompTIA Certification Guides,Na
4,(ISC)2 CISSP Certified Information Systems Sec...,James Michael Stewart,4.8 out of 5,885 ratings,$61.99,Na,Programming,"Publisher : Sybex; 3rd edition (July 7, 2021)\...",Na,#1 Best Seller,in Computer Network Security,Na
...,...,...,...,...,...,...,...,...,...,...,...,...
395,"Programming in Lua, fourth edition",Roberto Ierusalimschy,4.6 out of 5,322 ratings,$28.99,Na,Computer Science,"Publisher : Lua.Org (August 1, 2016)\nLanguage...",Na,Na,Na,Na
396,Adobe Photoshop Lightroom Classic Classroom in...,Rafael Concepcion,4.5 out of 5,55 ratings,$63.13,Na,Graphics & Design,Publisher : Adobe Press; 1st edition (March 14...,Na,Na,Na,Part of: Classroom in a Book (142 books)
397,Logic Pro - Apple Pro Training Series: Profess...,David Nahmani,4.5 out of 5,10 ratings,$59.03,Na,Computer Science,Publisher : Peachpit Press; 1st edition (May 1...,Na,Na,Na,Part of: Apple Pro Training (63 books)
398,AWS Cookbook: Recipes for Success on AWS,John Culkin,4.6 out of 5,123 ratings,$46.99,Na,Software,Publisher : O'Reilly Media; 1st edition (Decem...,Na,Na,Na,Na


In [19]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

In [36]:
result_2 = get_details(d2)
result_2.to_csv('result_2.csv')
driver.quit()
result_2

,book_name,author,rating_value,rating_num,price_buy_new,price_rent,category,details,rank,tag,tag_in,part_in
0,The Kaggle Book: Data analysis and machine lea...,Konrad Banachewicz,4.5 out of 5,99 ratings,$46.99,Na,Computer Science,"Publisher : Packt Publishing (April 22, 2022)\...",Na,Na,Na,Na
1,"Product Management in Practice: A Practical, T...",Matt LeMay,4.8 out of 5,39 ratings,$32.99,Na,Management & Leadership,Publisher : O'Reilly Media; 2nd edition (June ...,Na,Na,Na,Na
2,Version Control with Git: Powerful Tools and T...,Prem Ponuthorai,5 out of 5,1 rating,$48.49,Na,Networking & Cloud Computing,Publisher : O'Reilly Media; 3rd edition (Novem...,Na,Na,Na,Na
3,Data Analysis for Social Science: A Friendly a...,Elena Llaudet,4.8 out of 5,8 ratings,$42.99,Na,Mathematics,Publisher : Princeton University Press (Novemb...,Na,Na,Na,Na
4,CISA Certified Information Systems Auditor All...,Peter H. Gregory,4.6 out of 5,193 ratings,$47.53,Na,Networking & Cloud Computing,Publisher : McGraw Hill; 4th edition (November...,,Na,Na,Na
...,...,...,...,...,...,...,...,...,...,...,...,...
395,SHRM-CP/SHRM-SCP Certification Practice Exams,William Kelly,4.4 out of 5,171 ratings,$31.93,Na,Human Resources,ASIN : 1259584887\nPublisher : McGraw Hill; 1s...,Na,Na,Na,Na
396,Beginning Programming All-in-One For Dummies,Wallace Wang,4.6 out of 5,15 ratings,$29.49,Na,Programming,"Publisher : For Dummies; 2nd edition (June 3, ...",,Na,Na,Na
397,Microsoft Project For Dummies (For Dummies (Co...,Cynthia Snyder Dionisio,4.6 out of 5,30 ratings,$23.99,Na,Management & Leadership,Publisher : For Dummies; 1st edition (January ...,Na,Na,Na,Na
398,Practical Natural Language Processing: A Compr...,Sowmya Vajjala,4.2 out of 5,149 ratings,$55.01,Na,Computer Science,Publisher : O'Reilly Media; 1st edition (July ...,Na,Na,Na,Na


In [21]:
result_3 = get_details(d3)
result_3.to_csv('result_3.csv')
driver.quit()
result_3

,book_name,author,rating_value,rating_num,price_buy_new,price_rent,category,details,rank,tag,tag_in,part_in
0,"Unit Testing Principles, Practices, and Patter...",Vladimir Khorikov,4.7 out of 5,169 ratings,$49.99,Na,Programming,"Publisher : Manning; 1st edition (January 14, ...",Na,Na,Na,Na
1,SQL: Learn SQL (using MySQL) in One Day and Le...,LCF Publishing,4.5 out of 5,912 ratings,Na,Na,Computers & Technology,ASIN : B07K374J19\nPublication date : October ...,Na,#1 Best Seller,in Software Engineering,Book 5 of 7: Learn Coding Fast with Hands-On P...
2,Container Security: Fundamental Technology Con...,Liz Rice,4.6 out of 5,228 ratings,$35.49,Na,Networking & Cloud Computing,Publisher : O'Reilly Media; 1st edition (April...,,Na,Na,Na
3,Blender 3D By Example: A project-based guide t...,Oscar Baechler,4.4 out of 5,166 ratings,$33.49,Na,Computer Science,Publisher : Packt Publishing; Illustrated edit...,Na,#1 Best Seller,in Rendering & Ray Tracing,Na
4,AWS Certified Cloud Practitioner Practice Tests,Neal Davis,4.7 out of 5,561 ratings,$24.99,Na,Networking & Cloud Computing,ASIN : B08F6LDVD3\nPublisher : Independently p...,Na,Na,Na,Na
...,...,...,...,...,...,...,...,...,...,...,...,...
391,Artificial Intelligence Programming with Pytho...,Perry Xiao,5 out of 5,11 ratings,$28.49,Na,Computer Science,"Publisher : Wiley; 1st edition (March 11, 2022...",Na,Na,Na,Na
392,Fundamentals of Biostatistics,Bernard Rosner,4.2 out of 5,190 ratings,$132.97,$41.27,Biological Sciences,Publisher : Cengage Learning; 8th edition (Aug...,,Na,Na,Na
393,Applied Predictive Modeling,Max Kuhn,4.6 out of 5,297 ratings,$64.67,$38.99,Biological Sciences,"Publisher : Springer; 1st ed. 2013, Corr. 2nd ...",,Na,Na,Na
394,An Introduction to Mathematical Cryptography (...,Jeffrey Hoffstein,4.6 out of 5,45 ratings,$72.69,Na,Mathematics,ASIN : 1493917102\nPublisher : Springer; 2nd e...,Na,Na,Na,Part of: Undergraduate Texts in Mathematics (1...


### Merge data

In [3]:
res1 = pd.read_csv('result_1.csv')
res2 = pd.read_csv('result_2.csv')
res3 = pd.read_csv('result_3.csv')
res = pd.concat([ res1, res2, res3 ], axis =0)
res

,Unnamed: 0,book_name,author,rating_value,rating_num,price_buy_new,price_rent,category,details,rank,tag,tag_in,part_in
0,0,Cracking the Coding Interview: 189 Programming...,Gayle Laakmann McDowell,4.7 out of 5,"8,162 ratings",$35.95,Na,Programming,ASIN : 0984782850\nPublisher : CareerCup; 6th ...,NaN,#1 Best Seller,in Job Interviewing,Part of: Cracking the Interview & Career (3 bo...
1,1,Designing Data-Intensive Applications: The Big...,Martin Kleppmann,4.8 out of 5,"3,663 ratings",$47.94,Na,Databases & Big Data,Publisher : O'Reilly Media; 1st edition (May 2...,Na,#1 Best Seller,in MySQL Guides,Na
2,2,Machine Learning System Design Interview,Ali Aminian,4.5 out of 5,10 ratings,$38.00,Na,Computer Science,"Publisher : ByeByteGo (January 28, 2023)\nLang...",Na,#1 Best Seller,in Computer Science,Na
3,3,CompTIA A+ Certification All-in-One Exam Guide...,Travis Everett,4.3 out of 5,23 ratings,$54.00,Na,Hardware & DIY,Publisher : McGraw Hill; 11th edition (January...,Na,#1 Best Seller,in CompTIA Certification Guides,Na
4,4,(ISC)2 CISSP Certified Information Systems Sec...,James Michael Stewart,4.8 out of 5,885 ratings,$61.99,Na,Programming,"Publisher : Sybex; 3rd edition (July 7, 2021)\...",Na,#1 Best Seller,in Computer Network Security,Na
...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,391,Artificial Intelligence Programming with Pytho...,Perry Xiao,5 out of 5,11 ratings,$28.49,Na,Computer Science,"Publisher : Wiley; 1st edition (March 11, 2022...",Na,Na,Na,Na
392,392,Fundamentals of Biostatistics,Bernard Rosner,4.2 out of 5,190 ratings,$132.97,$41.27,Biological Sciences,Publisher : Cengage Learning; 8th edition (Aug...,NaN,Na,Na,Na
393,393,Applied Predictive Modeling,Max Kuhn,4.6 out of 5,297 ratings,$64.67,$38.99,Biological Sciences,"Publisher : Springer; 1st ed. 2013, Corr. 2nd ...",NaN,Na,Na,Na
394,394,An Introduction to Mathematical Cryptography (...,Jeffrey Hoffstein,4.6 out of 5,45 ratings,$72.69,Na,Mathematics,ASIN : 1493917102\nPublisher : Springer; 2nd e...,Na,Na,Na,Part of: Undergraduate Texts in Mathematics (1...


In [24]:
res.to_csv('res.csv') # data đã crawl, ko bao gồm asin

In [10]:
df1_asin = df1_asin.rename(columns={'titles': 'book_name'})

,Unnamed: 0,index,book_name,urls,asin,asin_url
0,0,0,Cracking the Coding Interview: 189 Programming...,https://www.amazon.com/Cracking-Coding-Intervi...,0984782850,https://www.amazon.com/dp/0984782850/
1,1,1,Designing Data-Intensive Applications: The Big...,https://www.amazon.com/Designing-Data-Intensiv...,1449373321,https://www.amazon.com/dp/1449373321/
2,2,2,Machine Learning System Design Interview,https://www.amazon.com/Machine-Learning-System...,1736049127,https://www.amazon.com/dp/1736049127/
3,3,3,CompTIA A+ Certification All-in-One Exam Guide...,https://www.amazon.com/CompTIA-Certification-E...,1264609906,https://www.amazon.com/dp/1264609906/
4,4,4,(ISC)2 CISSP Certified Information Systems Sec...,https://www.amazon.com/Certified-Information-S...,1119790026,https://www.amazon.com/dp/1119790026/
...,...,...,...,...,...,...
1191,1191,1195,Artificial Intelligence Programming with Pytho...,https://www.amazon.com/Artificial-Intelligence...,1119820863,https://www.amazon.com/dp/1119820863/
1192,1192,1196,Fundamentals of Biostatistics,https://www.amazon.com/Fundamentals-Biostatist...,130526892X,https://www.amazon.com/dp/130526892X/
1193,1193,1197,Applied Predictive Modeling,https://www.amazon.com/Applied-Predictive-Mode...,1461468485,https://www.amazon.com/dp/1461468485/
1194,1194,1198,An Introduction to Mathematical Cryptography (...,https://www.amazon.com/Introduction-Mathematic...,1493917102,https://www.amazon.com/dp/1493917102/


In [11]:
df1_asin.head()

,Unnamed: 0,index,book_name,urls,asin,asin_url
0,0,0,Cracking the Coding Interview: 189 Programming...,https://www.amazon.com/Cracking-Coding-Intervi...,0984782850,https://www.amazon.com/dp/0984782850/
1,1,1,Designing Data-Intensive Applications: The Big...,https://www.amazon.com/Designing-Data-Intensiv...,1449373321,https://www.amazon.com/dp/1449373321/
2,2,2,Machine Learning System Design Interview,https://www.amazon.com/Machine-Learning-System...,1736049127,https://www.amazon.com/dp/1736049127/
3,3,3,CompTIA A+ Certification All-in-One Exam Guide...,https://www.amazon.com/CompTIA-Certification-E...,1264609906,https://www.amazon.com/dp/1264609906/
4,4,4,(ISC)2 CISSP Certified Information Systems Sec...,https://www.amazon.com/Certified-Information-S...,1119790026,https://www.amazon.com/dp/1119790026/


In [14]:
result = pd.merge(res, df1_asin[['asin', 'asin_url', 'book_name']], on ='book_name')
result

,Unnamed: 0,book_name,author,rating_value,rating_num,price_buy_new,price_rent,category,details,rank,tag,tag_in,part_in,asin,asin_url
0,0,Cracking the Coding Interview: 189 Programming...,Gayle Laakmann McDowell,4.7 out of 5,"8,162 ratings",$35.95,Na,Programming,ASIN : 0984782850\nPublisher : CareerCup; 6th ...,NaN,#1 Best Seller,in Job Interviewing,Part of: Cracking the Interview & Career (3 bo...,0984782850,https://www.amazon.com/dp/0984782850/
1,1,Designing Data-Intensive Applications: The Big...,Martin Kleppmann,4.8 out of 5,"3,663 ratings",$47.94,Na,Databases & Big Data,Publisher : O'Reilly Media; 1st edition (May 2...,Na,#1 Best Seller,in MySQL Guides,Na,1449373321,https://www.amazon.com/dp/1449373321/
2,2,Machine Learning System Design Interview,Ali Aminian,4.5 out of 5,10 ratings,$38.00,Na,Computer Science,"Publisher : ByeByteGo (January 28, 2023)\nLang...",Na,#1 Best Seller,in Computer Science,Na,1736049127,https://www.amazon.com/dp/1736049127/
3,3,CompTIA A+ Certification All-in-One Exam Guide...,Travis Everett,4.3 out of 5,23 ratings,$54.00,Na,Hardware & DIY,Publisher : McGraw Hill; 11th edition (January...,Na,#1 Best Seller,in CompTIA Certification Guides,Na,1264609906,https://www.amazon.com/dp/1264609906/
4,4,(ISC)2 CISSP Certified Information Systems Sec...,James Michael Stewart,4.8 out of 5,885 ratings,$61.99,Na,Programming,"Publisher : Sybex; 3rd edition (July 7, 2021)\...",Na,#1 Best Seller,in Computer Network Security,Na,1119790026,https://www.amazon.com/dp/1119790026/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,390,Introduction to Networks Labs and Study Guide ...,Allan Johnson,4.5 out of 5,50 ratings,$60.00,Na,Networking & Cloud Computing,"Publisher : Cisco Press; 1st edition (June 17,...",Na,Na,Na,Na,0136634451,https://www.amazon.com/dp/0136634451/
1222,391,Artificial Intelligence Programming with Pytho...,Perry Xiao,5 out of 5,11 ratings,$28.49,Na,Computer Science,"Publisher : Wiley; 1st edition (March 11, 2022...",Na,Na,Na,Na,1119820863,https://www.amazon.com/dp/1119820863/
1223,392,Fundamentals of Biostatistics,Bernard Rosner,4.2 out of 5,190 ratings,$132.97,$41.27,Biological Sciences,Publisher : Cengage Learning; 8th edition (Aug...,NaN,Na,Na,Na,130526892X,https://www.amazon.com/dp/130526892X/
1224,393,Applied Predictive Modeling,Max Kuhn,4.6 out of 5,297 ratings,$64.67,$38.99,Biological Sciences,"Publisher : Springer; 1st ed. 2013, Corr. 2nd ...",NaN,Na,Na,Na,1461468485,https://www.amazon.com/dp/1461468485/


In [22]:
duplicate_rows = result[result.duplicated(['asin'])]
len(duplicate_rows)

32

In [27]:
data = result.drop_duplicates(subset='asin')
data

,Unnamed: 0,book_name,author,rating_value,rating_num,price_buy_new,price_rent,category,details,rank,tag,tag_in,part_in,asin,asin_url
0,0,Cracking the Coding Interview: 189 Programming...,Gayle Laakmann McDowell,4.7 out of 5,"8,162 ratings",$35.95,Na,Programming,ASIN : 0984782850\nPublisher : CareerCup; 6th ...,NaN,#1 Best Seller,in Job Interviewing,Part of: Cracking the Interview & Career (3 bo...,0984782850,https://www.amazon.com/dp/0984782850/
1,1,Designing Data-Intensive Applications: The Big...,Martin Kleppmann,4.8 out of 5,"3,663 ratings",$47.94,Na,Databases & Big Data,Publisher : O'Reilly Media; 1st edition (May 2...,Na,#1 Best Seller,in MySQL Guides,Na,1449373321,https://www.amazon.com/dp/1449373321/
2,2,Machine Learning System Design Interview,Ali Aminian,4.5 out of 5,10 ratings,$38.00,Na,Computer Science,"Publisher : ByeByteGo (January 28, 2023)\nLang...",Na,#1 Best Seller,in Computer Science,Na,1736049127,https://www.amazon.com/dp/1736049127/
3,3,CompTIA A+ Certification All-in-One Exam Guide...,Travis Everett,4.3 out of 5,23 ratings,$54.00,Na,Hardware & DIY,Publisher : McGraw Hill; 11th edition (January...,Na,#1 Best Seller,in CompTIA Certification Guides,Na,1264609906,https://www.amazon.com/dp/1264609906/
4,4,(ISC)2 CISSP Certified Information Systems Sec...,James Michael Stewart,4.8 out of 5,885 ratings,$61.99,Na,Programming,"Publisher : Sybex; 3rd edition (July 7, 2021)\...",Na,#1 Best Seller,in Computer Network Security,Na,1119790026,https://www.amazon.com/dp/1119790026/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,390,Introduction to Networks Labs and Study Guide ...,Allan Johnson,4.5 out of 5,50 ratings,$60.00,Na,Networking & Cloud Computing,"Publisher : Cisco Press; 1st edition (June 17,...",Na,Na,Na,Na,0136634451,https://www.amazon.com/dp/0136634451/
1222,391,Artificial Intelligence Programming with Pytho...,Perry Xiao,5 out of 5,11 ratings,$28.49,Na,Computer Science,"Publisher : Wiley; 1st edition (March 11, 2022...",Na,Na,Na,Na,1119820863,https://www.amazon.com/dp/1119820863/
1223,392,Fundamentals of Biostatistics,Bernard Rosner,4.2 out of 5,190 ratings,$132.97,$41.27,Biological Sciences,Publisher : Cengage Learning; 8th edition (Aug...,NaN,Na,Na,Na,130526892X,https://www.amazon.com/dp/130526892X/
1224,393,Applied Predictive Modeling,Max Kuhn,4.6 out of 5,297 ratings,$64.67,$38.99,Biological Sciences,"Publisher : Springer; 1st ed. 2013, Corr. 2nd ...",NaN,Na,Na,Na,1461468485,https://www.amazon.com/dp/1461468485/


In [28]:
data.to_csv('data.csv')